In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
automate_file = "Resources/automation_data_by_state.csv"
salary_file = "Resources/occupation_salary.csv"
soc_file = "Resources/soc_codes.csv"

### Cleaning Automation Data 

In [3]:
automation_df = pd.read_csv(automate_file)
automation_df.rename(columns = {
    'District of Columbia':'District_of_Columbia',
    'New Hampshire':'New_Hampshire',
    'New Jersey':'New_Jersey','New Mexico':'New_Mexico',
    'New York':'New_York','North Carolina':'North_Carolina',
    'North Dakota':'North_Dakota','Rhode Island':'Rhode_Island',
    'South Carolina':'South_Carolina',
    'South Dakota':'South_Dakota','West Virginia':'West_Virginia',
    'Occupation':'OCCUPATION','Probability':'Automation_Probability'
},inplace=True)

automation_df.drop(columns=['OCCUPATION'],inplace=True)

In [4]:
automation_df.head()

,SOC,Automation_Probability,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,...,South_Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West_Virginia,Wisconsin,Wyoming
0,11-1011,0.015,1030,760,5750,2710,31150,880,1410,340,...,560,5460,5890,3650,280,6320,5910,980,3740,160
1,11-1021,0.160,26930,6490,43300,20680,261780,41540,33280,4080,...,3730,44400,168610,36200,2760,52380,43760,10200,32350,4840
2,11-2011,0.039,50,40,470,110,3760,480,300,0,...,0,670,1210,380,40,240,640,40,200,0
3,11-2021,0.014,530,200,4790,1090,33390,3060,4970,590,...,60,3400,9570,2320,380,3840,5830,260,2980,30
4,11-2022,0.013,2510,400,10650,2650,69180,4570,7040,860,...,300,8890,22310,3360,480,5360,7390,600,5730,180


### Cleaning Salary Data 

In [5]:
salary_df = pd.read_csv(salary_file)
salary_df = salary_df.rename(columns={'ANNUAL':'ANNUAL_only', 
                                      "HOURLY":"HOURLY_only",
                                      'OCC_CODE':'SOC'})

bool_list = ['ANNUAL_only','HOURLY_only']

salary_df[bool_list] = salary_df[bool_list].fillna(0).astype(int)

salary_df[bool_list] = salary_df[bool_list].replace(True,1).astype(int)

salary_df[bool_list] = salary_df[bool_list].astype(bool)

In [6]:
salary_df.replace(",","", regex=True,inplace=True)

salary_df = salary_df.replace('*', None)
salary_df = salary_df.replace('**', None)
salary_df = salary_df.replace('#', None)

salary_df.drop(columns=['OCC_TITLE'],inplace=True)

In [7]:
integer_list = ['TOT_EMP','EMP_PRSE','H_MEAN','A_MEAN','MEAN_PRSE','H_PCT10',
                'H_PCT25','H_MEDIAN','H_PCT75','H_PCT90','A_PCT10','A_PCT25',
                'A_MEDIAN','A_PCT75','A_PCT90','ANNUAL_only','HOURLY_only']

salary_df[integer_list] = salary_df[integer_list].astype(float)

# Drop row for Total employment (SOC: 00-0000)
salary_df = salary_df[salary_df['SOC'] != '00-0000']

In [8]:
salary_df.head()

,SOC,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL_only,HOURLY_only
1,11-0000,major,7090790.0,0.2,56.74,118020.0,0.1,22.76,32.99,48.46,70.72,45.45,47330.0,68630.0,100790.0,147090.0,94540.0,0.0,0.0
2,11-1000,minor,2465800.0,0.2,61.03,126950.0,0.2,20.58,31.45,49.19,78.35,45.45,42810.0,65420.0,102320.0,162970.0,94540.0,0.0,0.0
3,11-1010,broad,223260.0,0.7,93.44,194350.0,0.4,33.55,54.86,87.12,78.35,45.45,69780.0,114100.0,181210.0,162970.0,94540.0,0.0,0.0
4,11-1011,detailed,223260.0,0.7,93.44,194350.0,0.4,33.55,54.86,87.12,78.35,45.45,69780.0,114100.0,181210.0,162970.0,94540.0,0.0,0.0
5,11-1020,broad,2188870.0,0.3,58.70,122090.0,0.2,21.29,31.20,47.74,74.53,45.45,44290.0,64890.0,99310.0,155020.0,94540.0,0.0,0.0


### Cleaning up SOC code data

In [9]:
soc_df = pd.read_csv(soc_file)
soc_df.head()

,SOC Group,SOC Code,SOC Title
0,Major,11-0000,Management Occupations
1,Minor,11-1000,Top Executives
2,Broad,11-1010,Chief Executives
3,Detailed,11-1011,Chief Executives
4,Broad,11-1020,General and Operations Managers


In [10]:
soc_df = pd.read_csv(soc_file)
soc_df.rename(columns={'SOC Code':'SOC_code',
                       'SOC Title':'SOC_title',
                      'SOC Group':'SOC_group'},
                      inplace=True)

In [11]:
# ERROR - missing key for this role in Primary Key list
soc_df.loc[soc_df['SOC_code']=='13-1078',:]

,SOC_group,SOC_code,SOC_title
1545,NaN,13-1078,Human Resources; Training; and Labor Relations...


## Create database connection 

In [12]:
conn = 'postgres:postgres@localhost:5432/labor_stats_db'
engine = create_engine(f'postgresql://{conn}')

In [13]:
soc_df.to_sql(name='SOC_keys',con=engine,if_exists='append',index=False)

In [14]:
salary_df.to_sql(name='Salary',con=engine,if_exists='append',index=False)

In [15]:
automation_df.to_sql(name='Automation',con=engine,if_exists='append',index=False)